# Clean the data

In [44]:
import yaml
import os
import pandas as pd

In [45]:
def get_config(config_file):
    ''' open config file with name config_file that contains parameters
    for this module and return Python object

    Args:
        config_file: filename containing config parameters

    Returns:
        config: Python dictionary with config parms from config file - dictionary


    '''
    current_path = os.getcwd()
    path_to_yaml = os.path.join(current_path, config_file)
    print("path_to_yaml " + path_to_yaml)
    try:
        with open(path_to_yaml, 'r') as c_file:
            config = yaml.safe_load(c_file)
        return config
    except Exception as error:
        print('Error reading the config file ' + str(error))

In [46]:
def print_config_values(config):
    for val in config:
        print("config value ",val," ",str(config[val]))

In [47]:
# read in pickle file
df = pd.read_pickle('data/data.pkl')

df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


### Determine appropriate replacement values for each column
filling in missing values with mean or median can reduce the variability of the data, while filling in with a placeholder value like 'Unknown' can introduce a new category in the data.

**Categorical Columns**:

- neighbourhood_group: This is a categorical column with values indicating the neighbourhood group of the Airbnb listing. A possible strategy for filling missing values in this column could be to use the mode (most frequently occurring value). However, filling with a specific value like 'Unknown' may be more appropriate as it clearly indicates that the data was missing.

- neighbourhood: Similar to 'neighbourhood_group', this column contains categorical values indicating the neighbourhood of the Airbnb listing. The same strategies mentioned for 'neighbourhood_group' can apply here.

- room_type: This column contains values indicating the type of room offered. Again, the mode or a value like 'Unknown' could be used to fill missing values.

**Continuous Columns**:

- latitude, longitude: These columns should ideally not have missing values as they are crucial for identifying the location of the listing. If they are missing, it could indicate an error during data collection. Since it's not appropriate to replace missing latitude or longitude with an average or median value, we may consider dropping rows with missing values in these columns or replacing them with a placeholder value that indicates an unknown location.
- price, minimum_nights, number_of_reviews, reviews_per_month, calculated_host_listings_count, availability_365: For these columns, missing values could be replaced with the mean, median, or mode of the column. However, this decision should be based on the distribution of the data. If the data is skewed, the median might be a better choice than the mean. This could lead to less variability in the data. Another strategy is to use a value that is outside the normal range of the variable, such as -9999. This is based on the idea that the model will learn that this value is different and represents missing data.

**Text Columns**:

name, host_name: These are text columns and it's likely that a missing value indicates that the information was not provided. A simple strategy here could be to replace missing values with a placeholder text such as 'Unknown'.

In [48]:
# Count of missing values for each column
for column in df.columns:
    print("Missing values in '{}' : {}".format(column, df[column].isnull().sum()))


Missing values in 'id' : 0
Missing values in 'name' : 16
Missing values in 'host_id' : 0
Missing values in 'host_name' : 21
Missing values in 'neighbourhood_group' : 0
Missing values in 'neighbourhood' : 0
Missing values in 'latitude' : 0
Missing values in 'longitude' : 0
Missing values in 'room_type' : 0
Missing values in 'price' : 0
Missing values in 'minimum_nights' : 0
Missing values in 'number_of_reviews' : 0
Missing values in 'last_review' : 10052
Missing values in 'reviews_per_month' : 10052
Missing values in 'calculated_host_listings_count' : 0
Missing values in 'availability_365' : 0


- The 'name' and 'host_name' will be replaced with 'Unknown'.

In [49]:
# Define function to replace missing values in the text columns with 'Unknown'
def replace_missing_val_text(df, col_list):
    '''
    Function to replace missing values in the text columns with 'Unknown'

    Parameters:
    df: dataframe
    col_list: list of columns to replace missing values

    Returns:
    df: dataframe with missing values replaced with 'Unknown'
    '''
    for col in col_list:
        df[col] = df[col].fillna('Unknown')
    return df

In [50]:
# call the function to replace missing values in the columns 'name' and 'host_name'
df = replace_missing_val_text(df,['name','host_name'])

In [53]:
# Count of missing values for each column
for column in df.columns:
    print("Missing values in '{}' : {}".format(column, df[column].isnull().sum()))

Missing values in 'id' : 0
Missing values in 'name' : 0
Missing values in 'host_id' : 0
Missing values in 'host_name' : 0
Missing values in 'neighbourhood_group' : 0
Missing values in 'neighbourhood' : 0
Missing values in 'latitude' : 0
Missing values in 'longitude' : 0
Missing values in 'room_type' : 0
Missing values in 'price' : 0
Missing values in 'minimum_nights' : 0
Missing values in 'number_of_reviews' : 0
Missing values in 'last_review' : 10052
Missing values in 'reviews_per_month' : 10052
Missing values in 'calculated_host_listings_count' : 0
Missing values in 'availability_365' : 0


- For the last_review column i will turn it into a numeric column representing the number of days since a specific reference date can provide a continuous variable that may be more meaningful for a machine learning model

In [54]:
def create_reviewed_indicator(df, column='last_review'):
    """
    Function to create a new boolean column that indicates whether a listing 
    has been reviewed or not (1 if 'last_review' is not null, 0 if 'last_review' is null).

    Parameters:
    - df: DataFrame that includes the 'last_review' column
    - column: Name of the column to check for null values. Default is 'last_review'.

    Returns:
    - DataFrame with the new 'reviewed' column.
    """

    df['reviewed'] = df[column].notnull().astype(int)

    return df



In [55]:
df = create_reviewed_indicator(df, column='last_review')

In [59]:
def transform_last_review(df, column='last_review', missing_value=99999):
    """
    Function to transform 'last_review' column to a numerical value representing 
    the number of days since a particular reference date. 

    Parameters:
    - df: DataFrame that includes the 'last_review' column
    - column: Name of the column to transform. Default is 'last_review'.
    - missing_value: Value to replace missing values with. Default is 99999.

    Returns:
    - DataFrame with the transformed 'last_review' column, renamed as 'days_since_last_review'.
    """

    # Ensure 'last_review' is in datetime format
    df[column] = pd.to_datetime(df[column])

    # Use the max date in 'last_review' as reference
    reference_date = df[column].max()

    # Calculate the number of days since last review and replace the 'last_review' column
    df[column] = (reference_date - df[column]).dt.days

    # Fill missing values with a specified large number
    df[column].fillna(missing_value, inplace=True)

    # Rename the column to 'days_since_last_review'
    df.rename(columns={column: 'days_since_last_review'}, inplace=True)

    return df


In [60]:
df = transform_last_review(df)

In [61]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,days_since_last_review,reviews_per_month,calculated_host_listings_count,availability_365,reviewed
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,262.0,0.21,6,365,1
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,48.0,0.38,2,355,1
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,99999.0,NaN,1,365,0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,3.0,4.64,1,194,1
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,231.0,0.10,1,0,1


- assuming if the 'reviews_per_month' field is missing, it means that the listing did not receive any reviews. We will replace the missing 'reviews_per_month' values with 0.

In [62]:
df['reviews_per_month'].fillna(0, inplace=True)

In [71]:
def check_missing_values(df):
    missing_values = False
    for column in df.columns:
        missing_count = df[column].isnull().sum()
        if missing_count > 0:
            print(f"Missing values in '{column}' : {missing_count}")
            missing_values = True
    if not missing_values:
        print("No missing values left!")
        
check_missing_values(df)


No missing values left!


In [73]:
# save the dataframe to pickle file
df.to_pickle('data/data.pkl')